In [126]:
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
from keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from numpy import array
from keras.models import Sequential
from keras.layers import Dense, Dropout, ConvLSTM2D, Flatten, Conv2D, Input ,LSTM
from keras.layers import TimeDistributed
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.optimizers import Adam

In [106]:
datapath = 'https://raw.githubusercontent.com/JonathanC1/Datasets/master/TorNonTor/SelectedFeatures-10s-TOR-NonTOR.csv'
dataframe = pd.read_csv(datapath,low_memory=False)

In [107]:
def dfNormalize(df):
    for feature_name in df.columns:
        df.loc[:,feature_name]= pd.to_numeric(df.loc[:,feature_name], errors='coerce').fillna(0)
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()   
        if (max_value - min_value) > 0:
            df.loc[:,feature_name] = (df.loc[:,feature_name] - min_value) / (max_value - min_value)
        else:
            df.loc[:,feature_name] = (df.loc[:,feature_name]- min_value)    
    return df

In [108]:
print(dataframe.shape)
dataframe = dataframe.reindex(np.random.permutation(dataframe.index)).copy()
print(dataframe.describe())
print(list(dataframe))

(16088, 29)
        Source Port   Destination Port   Protocol   Flow Duration  \
count  16088.000000       16088.000000    16088.0    1.608800e+04   
mean   36296.304388       11777.729426        6.0    4.422426e+06   
std    21217.827170       19907.298714        0.0    4.650899e+06   
min       21.000000          21.000000        6.0    1.000000e+00   
25%    33224.000000         443.000000        6.0    3.710925e+04   
50%    41282.000000         443.000000        6.0    7.780915e+05   
75%    54420.000000        9001.000000        6.0    9.965477e+06   
max    65087.000000       65514.000000        6.0    1.000000e+07   

        Flow IAT Max   Flow IAT Min   Fwd IAT Max   Fwd IAT Min   Bwd IAT Max  \
count   1.608800e+04   1.608800e+04  1.608800e+04  1.608800e+04  1.608800e+04   
mean    8.139237e+05   6.711377e+04  7.562887e+05  8.203885e+04  5.994034e+05   
std     1.775624e+06   4.228532e+05  1.745859e+06  6.422426e+05  1.525304e+06   
min     1.000000e+00  -2.255000e+03  0.000

In [109]:
keys = dataframe.keys()
data_to_process = dataframe[keys[4:len(keys) - 1]].copy()
x_normalised = dfNormalize(data_to_process)
print(x_normalised.describe())

        Protocol   Flow Duration   Flow Bytes/s   Flow Packets/s  \
count    16088.0    16088.000000   16088.000000     16088.000000   
mean         0.0        0.442243       0.000295         0.000620   
std          0.0        0.465090       0.012537         0.017354   
min          0.0        0.000000       0.000000         0.000000   
25%          0.0        0.003711       0.000000         0.000000   
50%          0.0        0.077809       0.000000         0.000000   
75%          0.0        0.996548       0.000000         0.000000   
max          0.0        1.000000       1.000000         1.000000   

        Flow IAT Mean   Flow IAT Std   Flow IAT Max   Flow IAT Min  \
count    16088.000000        16088.0   16088.000000   16088.000000   
mean         0.014998            0.0       0.081417       0.007002   
std          0.062858            0.0       0.177617       0.042685   
min          0.000000            0.0       0.000000       0.000000   
25%          0.000000            0.0 

In [110]:
change_labels = lambda x: 1 if x == 'nonTOR' else 0
y_normalised = dataframe['label'].apply(change_labels)

X_train, X_test, y_train, y_test = train_test_split(x_normalised, y_normalised, test_size = 0.20)

In [111]:
#definir el modelo
modelo = Sequential()
modelo.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)))
modelo.add(MaxPooling1D(pool_size=2))
modelo.add(LSTM(32))
modelo.add(Dense(1, activation='sigmoid'))
modelo.compile(optimizer=Adam(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])

C:\Users\ELEN\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [120]:
# Fit the model
modelo.fit(np.expand_dims(X_train.values, axis=2), y_train.values, batch_size=32, epochs=20, validation_data=(np.expand_dims(X_test.values, axis=2), y_test.values))

Epoch 1/20
403/403 [==============================] - 3s 8ms/step - loss: 0.0436 - accuracy: 0.9845 - val_loss: 0.0466 - val_accuracy: 0.9838
Epoch 2/20
403/403 [==============================] - 3s 7ms/step - loss: 0.0439 - accuracy: 0.9843 - val_loss: 0.0470 - val_accuracy: 0.9838
Epoch 3/20
403/403 [==============================] - 3s 7ms/step - loss: 0.0424 - accuracy: 0.9845 - val_loss: 0.0450 - val_accuracy: 0.9842
Epoch 4/20
403/403 [==============================] - 3s 7ms/step - loss: 0.0416 - accuracy: 0.9847 - val_loss: 0.0462 - val_accuracy: 0.9845
Epoch 5/20
403/403 [==============================] - 3s 7ms/step - loss: 0.0435 - accuracy: 0.9838 - val_loss: 0.0447 - val_accuracy: 0.9851
Epoch 6/20
403/403 [==============================] - 3s 7ms/step - loss: 0.0416 - accuracy: 0.9848 - val_loss: 0.0460 - val_accuracy: 0.9845
Epoch 7/20
403/403 [==============================] - 3s 7ms/step - loss: 0.0418 - accuracy: 0.9845 - val_loss: 0.0467 - val_accuracy: 0.9851
Epoch 

In [132]:
# Evaluate the model
score = modelo.evaluate(X_test, y_test, verbose=0)
#print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test accuracy: 0.9853946566581726
